In [1]:
import pandas as pd
import numpy as np
import fastf1 as ff1

In [2]:
# get the schedule df's by year

def get_dataframe_schedule(year):
    df = ff1.get_event_schedule(year)
    return (
        df
        .drop(columns=["Location", "OfficialEventName", "Session1Date", "Session1DateUtc", "Session2Date", "Session2DateUtc", "Session3Date", "Session3DateUtc",
                       "Session4Date", "Session4DateUtc", "Session5Date", "Session5DateUtc", "F1ApiSupport"])
        .loc[~df['EventName'].str.contains('Pre-Season', na=False)]
        .reset_index(drop=True)
    )

In [3]:
df1 = get_dataframe_schedule(2023)
df1.head()

req         WARNING 	DEFAULT CACHE ENABLED! (5.15 GB) C:\Users\Caio\AppData\Local\Temp\fastf1


,RoundNumber,Country,EventDate,EventName,EventFormat,Session1,Session2,Session3,Session4,Session5
0,1,Bahrain,2023-03-05,Bahrain Grand Prix,conventional,Practice 1,Practice 2,Practice 3,Qualifying,Race
1,2,Saudi Arabia,2023-03-19,Saudi Arabian Grand Prix,conventional,Practice 1,Practice 2,Practice 3,Qualifying,Race
2,3,Australia,2023-04-02,Australian Grand Prix,conventional,Practice 1,Practice 2,Practice 3,Qualifying,Race
3,4,Azerbaijan,2023-04-30,Azerbaijan Grand Prix,sprint_shootout,Practice 1,Qualifying,Sprint Shootout,Sprint,Race
4,5,United States,2023-05-07,Miami Grand Prix,conventional,Practice 1,Practice 2,Practice 3,Qualifying,Race


In [8]:
# get the race results df's

def get_race(year, gp):
    race = ff1.get_session(year, gp, "R")
    race.load(laps=False, telemetry=False, weather=False, messages=False, livedata=False)
    race_results_df = race.results
    race_results_df = race_results_df.drop(columns=["BroadcastName", "Abbreviation", "DriverId", "TeamColor", "TeamId", "FirstName", "LastName", "HeadshotUrl", "CountryCode", "Q1", "Q2", "Q3", "Time"])
    df_event = get_dataframe_schedule(year)
    event_name = df_event.at[gp-1, "EventName"]
    race_results_df["EventName"] = event_name
    return race_results_df

In [9]:
teste = get_race(2023, 2)

core           INFO 	Loading data for Saudi Arabian Grand Prix - Race [v3.3.9]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
Request for URL https://ergast.com/api/f1/2023/2/results.json failed; using cached response
Traceback (most recent call last):
  File "c:\Users\Caio\Desktop\apps\Project F1\venv\Lib\site-packages\requests_cache\session.py", line 291, in _resend
    response.raise_for_status()
  File "c:\Users\Caio\Desktop\apps\Project F1\venv\Lib\site-packages\requests\models.py", line 1024, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 503 Server Error: Backend fetch failed for url: https://ergast.com/api/f1/2023/2/results.json
core           INFO 	Finished loading data for 20 drivers: ['11', '1', '14', '63', '44', '55', '16', '31', '10', '20', '22', '27', '24', '21', '81', '2', '4', '77', '23', '18']


In [ ]:
# concat the results df's

race_dataframes = []
for race in range(1, df1["RoundNumber"].max() + 1):
    df_race = get_race(2023, race)
    race_dataframes.append(df_race)

df_race_concat = pd.concat(race_dataframes, axis=0, ignore_index=True)

In [5]:
# get laps data from race

def get_laps_data(year, gp):
    race = ff1.get_session(year, gp, "R")
    race.load(telemetry=False, weather=False, messages=False, livedata=False)
    df_laps = race.laps
    df = df_laps.drop(columns=["Time", "PitOutTime", "PitInTime", "Sector1SessionTime", "Sector2SessionTime", "Sector3SessionTime", "SpeedFL", "SpeedST",
                           "IsPersonalBest", "LapStartTime", "LapStartDate", "TrackStatus", "DeletedReason", "FastF1Generated", "IsAccurate"])
    return df

In [17]:
def get_weather_data(year, gp):
    weather = ff1.get_session(year, gp, "R")
    weather.load(telemetry=False, messages=False, livedata=False)
    df_weather = weather.laps.get_weather_data()
    df_weather = df_weather.drop(columns=["Humidity", "Pressure", "WindDirection", "WindSpeed"])
    return df_weather

In [ ]:
teste = get_weather_data(2023, 1)

In [19]:
teste

,Time,AirTemp,Rainfall,TrackTemp
62,0 days 01:02:45.730000,27.3,False,31.2
64,0 days 01:04:45.759000,27.3,False,31.1
66,0 days 01:06:45.757000,27.3,False,31.2
67,0 days 01:07:45.756000,27.3,False,31.2
69,0 days 01:09:45.754000,27.2,False,31.0
...,...,...,...,...
76,0 days 01:16:45.809000,27.2,False,30.8
78,0 days 01:18:45.822000,27.1,False,30.8
79,0 days 01:19:45.836000,27.2,False,30.6
81,0 days 01:21:45.834000,27.1,False,30.6


In [26]:
race = ff1.get_session(2023, 1, "R")
race.load(weather=False, messages=False, livedata=False)

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.3.9]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
Request for URL https://ergast.com/api/f1/2023/1/results.json failed; using cached response
Traceback (most recent call last):
  File "c:\Users\Caio\Desktop\apps\Project F1\venv\Lib\site-packages\requests_cache\session.py", line 291, in _resend
    response.raise_for_status()
  File "c:\Users\Caio\Desktop\apps\Project F1\venv\Lib\site-packages\requests\models.py", line 1024, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 503 Server Error: Backend fetch failed for url: https://ergast.com/api/f1/2023/1/results.json
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data


In [50]:
start_date = race.session_info["StartDate"]
start_date_timestamp = pd.Timestamp(start_date)

In [51]:
start_date_timestamp

Timestamp('2023-03-05 18:00:00')

In [37]:
telemetry = race.laps.pick_driver("VER").get_telemetry()

core        WARNING 	Failed to preserve data type for column 'X' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Y' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Z' while merging telemetry.


In [38]:
telemetry = telemetry.drop(columns=["Date", "DriverAhead", "DistanceToDriverAhead", "Source", "Distance", "RelativeDistance", "Status"])
telemetry

,SessionTime,Time,RPM,Speed,nGear,Throttle,Brake,DRS,X,Y,Z
2,0 days 01:02:36.652000,0 days 00:00:00,10100,0,1,16,False,1,-279.999631,3558.983052,-156.999885
3,0 days 01:02:36.762000,0 days 00:00:00.110000,10130,0,1,16,False,1,-279.999617,3558.982418,-156.999881
4,0 days 01:02:36.837000,0 days 00:00:00.185000,9948,0,1,16,False,1,-280.000000,3559.000000,-157.000000
5,0 days 01:02:36.962000,0 days 00:00:00.310000,9766,0,1,16,False,1,-280.001327,3559.060983,-157.000413
6,0 days 01:02:37.097000,0 days 00:00:00.445000,9206,0,1,16,False,1,-280.000000,3559.000000,-157.000000
...,...,...,...,...,...,...,...,...,...,...,...
42380,0 days 02:36:33.299000,0 days 01:33:56.647000,9971,257,7,0,True,0,-394.389121,984.144257,-159.000000
42381,0 days 02:36:33.336000,0 days 01:33:56.684000,9437,242,7,0,True,0,-393.000000,1010.000000,-159.000000
42382,0 days 02:36:33.659000,0 days 01:33:57.007000,8903,228,7,0,True,0,-382.456048,1224.383008,-159.000000
42383,0 days 02:36:33.676000,0 days 01:33:57.024000,8712,223,7,0,True,0,-382.000000,1235.000000,-159.000000


In [53]:
telemetry["Time"] = start_date_timestamp + pd.to_timedelta(telemetry["Time"])

In [55]:
telemetry.set_index('Time', inplace=True)

In [58]:
resampled_data = telemetry.resample('1s').agg({
    'RPM': 'mean',
    'Speed': 'mean',
    'Throttle': 'mean',
    'Brake': 'mean',
    'X': 'mean',
    'Y': 'mean',
    'Z': 'mean'
}).reset_index()

In [59]:
resampled_data

,Time,RPM,Speed,Throttle,Brake,X,Y,Z
0,2023-03-05 18:00:00,8697.111111,3.111111,16.000000,0.000000,-279.940578,3560.866992,-156.981553
1,2023-03-05 18:00:01,5061.777778,34.555556,22.888889,0.000000,-277.852467,3622.587074,-158.078033
2,2023-03-05 18:00:02,8579.142857,75.000000,60.571429,0.000000,-272.291164,3767.698060,-158.063964
3,2023-03-05 18:00:03,11837.500000,106.375000,88.125000,0.000000,-264.214239,3981.448717,-158.350138
4,2023-03-05 18:00:04,11345.666667,148.000000,100.000000,0.000000,-251.227163,4341.312059,-158.195231
...,...,...,...,...,...,...,...,...
5633,2023-03-05 19:33:53,11025.777778,239.666667,100.000000,0.000000,-519.685122,-1352.431353,-155.712240
5634,2023-03-05 19:33:54,11222.250000,256.125000,100.000000,0.000000,-480.920098,-643.261391,-158.425746
5635,2023-03-05 19:33:55,10661.000000,268.285714,100.000000,0.000000,-441.657396,76.635085,-159.000049
5636,2023-03-05 19:33:56,10282.142857,264.000000,7.142857,0.571429,-404.992918,781.063998,-159.000000
